In [1]:
%pip install -U -q google-genai

# Procesamiento de Lenguaje Natural con Google Gemini API
Este script demuestra diversas aplicaciones de Procesamiento de Lenguaje Natural (PLN) utilizando el modelo Gemini de Google a través de la librería `google-generativeai`.

In [2]:
from google.colab import userdata
import os

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai
from google.genai import types

cliente = genai.Client(api_key=GOOGLE_API_KEY)

# Definición del texto de entrada

In [4]:
# Texto base que utilizaremos para todos los ejemplos.

text_to_process = """Estimado Amazon, la semana pasada pedí una figura de acción de Optimus Prime
en su tienda en línea en Alemania. Desafortunadamente, cuando abrí el paquete,
descubrí con horror que me habían enviado una figura de acción de Megatron
en su lugar. Como enemigo de toda la vida de los Decepticons, espero que pueda
entender mi dilema. Para resolver el problema, exijo un cambio de Megatron por
la figura de Optimus Prime que pedí. Adjunto copias de mis registros relativos
a esta compra. Espero tener noticias suyas pronto. Atentamente, Bumblebee."""

print("\nTexto de entrada definido.")


Texto de entrada definido.


In [5]:
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

## 1. Sumarizacion

In [6]:
pregunta = f"""Sumariza el siguiente texto en dos oraciones de rapida lectura

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

Bumblebee le escribe a Amazon para quejarse de haber recibido una figura de acción de Megatron en vez de la de Optimus Prime que había ordenado. Exige que le cambien a Megatron por la figura correcta y adjunta sus comprobantes de compra.



##2. Clasificación de Sentimiento

In [7]:
pregunta = f"""Clasificá el siguiente texto como positivo, negativo o neutral y explicá por qué:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

El texto es **negativo**.

Aquí está la explicación:

*   **Tono general:** El tono del correo electrónico es de decepción y frustración. El uso de palabras como "desafortunadamente" y "con horror" denotan emociones negativas.

*   **Problema planteado:** El cliente recibió el producto incorrecto, lo cual es una experiencia negativa.

*   **Exigencia:** El cliente exige una solución al problema, lo cual indica insatisfacción.

Aunque el correo electrónico está escrito de manera cortés, el mensaje principal es sobre una experiencia negativa con un pedido.


##3. Reconocimiento de Entidades Nombradas (NER)

In [8]:
prompt = f"""Extraé todas las entidades nombradas del siguiente texto (personas, organizaciones, lugares, objetos) y clasificálas:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

Claro, aquí están las entidades nombradas extraídas del texto, clasificadas por tipo:

*   **Organizaciones:**
    *   Amazon
    *   Decepticons
*   **Personas:**
    *   Bumblebee
*   **Lugares:**
    *   Alemania
*   **Objetos:**
    *   Optimus Prime (figura de acción)
    *   Megatron (figura de acción)


##4. Respuesta a preguntas (Question Answering)

In [9]:
pregunta = "¿Qué producto recibió el cliente?"
contexto = text_to_process

prompt = f"""Respondé la siguiente pregunta basada en el texto:

Texto: {contexto}
Pregunta: {pregunta}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[contexto, pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)


El cliente recibió una figura de acción de Megatron.



##5. Resumen automático

In [10]:
prompt = f"""Resumí el siguiente texto en no más de 3 líneas:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)


Bumblebee recibió una figura de Megatron en lugar de la figura de Optimus Prime que había pedido a Amazon Alemania. Exige un cambio por el artículo correcto y adjuntó pruebas de su compra. Espera una pronta respuesta para solucionar el problema.



##6. Traducción (Español a Inglés)

In [11]:
prompt = f"""Traducí al inglés este texto:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)


Okay, here's the translation:

Dear Amazon,

Last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent a Megatron action figure instead. As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of the Megatron for the Optimus Prime figure that I ordered. I have attached copies of my records pertaining to this purchase. I look forward to hearing from you soon.

Sincerely,

Bumblebee.



##7. Generación de respuesta (como atención al cliente)

In [12]:
respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido. "

prompt = f"""{text_to_process}

Redactá una respuesta del servicio de atención al cliente que comience así:

"{respuesta_inicial}"

Cuya extension no supere las 4 lineas.
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

Estimado cliente, lamentamos mucho lo ocurrido con su pedido. Entendemos su frustración por recibir la figura de Megatron en lugar de la de Optimus Prime. Por favor, acepte nuestras disculpas. Procederemos a enviarle la figura correcta de Optimus Prime y le daremos instrucciones para devolver la figura de Megatron sin costo alguno para usted.



##8. Clasificación Zero-Shot (sin entrenamiento previo)

In [13]:
etiquetas = ["queja", "elogio", "consulta", "pedido", "agradecimiento"]

prompt = f"""Clasificá el siguiente texto en una de estas categorías: {', '.join(etiquetas)}. Justificá tu elección.

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)


**Clasificación: Queja**

**Justificación:**

El texto es claramente una **queja** por los siguientes motivos:

*   **Expresa insatisfacción:** Bumblebee manifiesta su "horror" al recibir un producto incorrecto (Megatron en lugar de Optimus Prime). Esto indica un sentimiento de decepción y descontento.
*   **Describe un problema:** El texto detalla un error en el pedido, especificando el producto incorrecto recibido.
*   **Solicita una solución:** Bumblebee "exige" un cambio del producto erróneo por el que originalmente ordenó, mostrando su intención de resolver el problema.
*   **Incluye evidencia:** La mención de adjuntar copias de los registros de compra refuerza la validez de la queja.

Todos estos elementos en conjunto señalan que el propósito principal del texto es expresar una inconformidad y buscar una solución a un error cometido por Amazon.



## EJERCICIO

Escribí un texto corto sobre una experiencia personal en un transporte público en Buenos Aires.

Luego, generá:

- Un resumen.
- Una clasificación de sentimiento.
- Una lista de entidades nombradas.

In [14]:
# Texto base que utilizaremos para todos los ejemplos.

text_to_process1 = """Tomé el colectivo 60 una mañana de lluvia.
Como siempre, venía lleno, y el parabrisas empañado hacía que el chofer manejara casi por instinto.
Me agarré como pude del caño, esquivando mochilas y paraguas cerrados que goteaban sobre el piso.
Una señora mayor me sonrió al ver que le cedí el asiento, y me agradeció con ese “gracias, nene” que te acomoda el día.
A pesar del apuro y el caos típico de la ciudad, hubo algo cálido en ese gesto mínimo. Un recordatorio de que, incluso en el bondi apretado, hay lugar para la humanidad."""

print("\nTexto de entrada definido.")


Texto de entrada definido.


# Resumen

In [17]:
prompt = f"""Resumí el siguiente texto en no más de 3 líneas:

Texto: {text_to_process1}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

En un colectivo 60 lleno y lluvioso, el narrador cede su asiento a una señora mayor. Su agradecimiento conmovió al narrador en medio del caos urbano. La experiencia demostró que la amabilidad persiste incluso en las situaciones más cotidianas.



# Clasificación de sentimiento

In [15]:
etiquetas = ["queja", "elogio", "consulta", "pedido", "agradecimiento"]

prompt = f"""Clasificá el siguiente texto en una de estas categorías: {', '.join(etiquetas)}. Justificá tu elección.

Texto: {text_to_process1}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

La categoría que mejor describe el texto es **elogio**.

**Justificación:**

Si bien el texto menciona aspectos negativos como el colectivo lleno, el parabrisas empañado y la incomodidad del viaje, el foco principal está en resaltar un aspecto positivo: la calidez humana. El autor enfatiza el gesto de la señora mayor y cómo ese simple agradecimiento tuvo un impacto positivo en su día.

*   **No es una queja:** Aunque menciona dificultades, no hay un reclamo directo o una expresión de insatisfacción hacia un servicio o persona en particular.
*   **No es una consulta:** No hay ninguna pregunta planteada en el texto.
*   **No es un pedido:** No se solicita nada al lector ni a ninguna entidad.
*   **No es un agradecimiento:** El autor no está agradeciendo a nadie en particular, sino que reflexiona sobre un acto de agradecimiento que recibió.

El texto celebra un acto de bondad y destaca la importancia de la humanidad en la vida cotidiana, lo que lo convierte en un elogio a la conexión huma

# Lista de entidades nombradas

In [16]:
prompt = f"""Extraé todas las entidades nombradas del siguiente texto (personas, organizaciones, lugares, objetos) y clasificálas:

Texto: {text_to_process1}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

*   **60:** (Objeto - Línea de colectivo)
*   **Bondi:** (Objeto - Colectivo)
*   **Ciudad:** (Lugar - Ciudad)
